In [2]:
import pandas as pd
import numpy as np
import re
import torch
from datasets import Dataset
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [3]:
dataset2 = pd.read_csv("/content/Customer_Support_Tweets.csv")

print(dataset2.head(6))

   tweet_id     author_id  inbound                      created_at  \
0    119237        105834     True  Wed Oct 11 06:55:44 +0000 2017   
1    119238  ChaseSupport    False  Wed Oct 11 13:25:49 +0000 2017   
2    119239        105835     True  Wed Oct 11 13:00:09 +0000 2017   
3    119240  VirginTrains    False  Tue Oct 10 15:16:08 +0000 2017   
4    119241        105836     True  Tue Oct 10 15:17:21 +0000 2017   
5    119243  VirginTrains    False  Tue Oct 10 15:25:14 +0000 2017   

                                                text response_tweet_id  \
0  @AppleSupport causing the reply to be disregar...            119236   
1  @105835 Your business means a lot to us. Pleas...               NaN   
2  @76328 I really hope you all change but I'm su...            119238   
3  @105836 LiveChat is online at the moment - htt...            119241   
4  @VirginTrains see attached error message. I've...            119243   
5  @105836 Have you tried from another device, Mi...            1

In [ ]:
print(dataset2.columns.tolist())

['tweet_id', 'author_id', 'inbound', 'created_at', 'text', 'response_tweet_id', 'in_response_to_tweet_id']


In [ ]:
dataset2 = dataset2[['text']].copy()

dataset2['urgency'] = np.random.choice([0, 1], size=len(dataset2))

dataset2.head()

,text,urgency
0,@AppleSupport causing the reply to be disregar...,0
1,@105835 Your business means a lot to us. Pleas...,0
2,@76328 I really hope you all change but I'm su...,1
3,@105836 LiveChat is online at the moment - htt...,0
4,@VirginTrains see attached error message. I've...,1


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

dataset2['clean_text'] = dataset2['text'].astype(str).apply(clean_text)


In [ ]:
from sklearn.model_selection import train_test_split


tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset2['clean_text'].tolist(), dataset2['urgency'].tolist(), test_size=0.2, random_state=42
)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10
)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

predictions = trainer.predict(test_dataset)

y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

# Evaluate
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f" Accuracy: {acc:.4f}")
print(f" F1 Score: {f1:.4f}")




 Accuracy: 0.3158
 F1 Score: 0.0000
